In [ ]:
%cd ..
%pwd

In [ ]:
from experiments.modeleval_utils import *

In [ ]:
# no feature selection
#output_dir = 'experiments/results'
#drop_columns = False # feature selection

# feature selection
output_dir = 'experiments/results_FS'
drop_columns = True 

In [ ]:
scoring = 'average_precision'
best_params_per_model = {}

for model in models:
    best_params_model = []
    for (data, target) in [('fixed_defect_szz', 'performance'), ('bugbug_buglevel', 'performance'), ('bugbug_buglevel', 'regression')]:
        for feature_type in ['traditional', 'bow']:
                try:
                    results = get_results(output_dir, data, feature_type, target, scoring, model)
                    params = get_params(results)
                    best_params = get_best_params(model, params)
                    del best_params['model']
                    best_params['sampler'] = str(best_params['sampler'])

                    index = {  
                        'Model': model_names[model],
                        'Data': data, 
                        'Feature Type': feature_type,
                        'Target': target
                        }
                    best_params_model.append({**index, **best_params})
                except FileNotFoundError:
                    pass
    if len(best_params_model) == 0:
        continue

    df = pd.DataFrame(best_params_model)
    df = df.replace({
                    'None': 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Random Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Random Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })

    df = df.replace({
        'fixed_defect_szz': 'SZZ',
        'bugbug_buglevel': 'bugbug'
    })
    df = df.replace({
        'performance': 'perf. regressions',
        'regression': 'all regressions',
        'bow': 'bag-of-words'
    })
    df.set_index(pd.MultiIndex.from_frame(df[['Data','Target', 'Feature Type',]]), inplace=True)
    df = df.drop(['Model','Data','Feature Type','Target'], axis=1)
    def rename_f(c):
        if 'model' in c:
            c = c[7:]
        if 'kernel' in c:
            c = c[8:]
        return c.capitalize()

    df = df.rename(rename_f, axis=1)
    best_params_per_model[model] = df
                

In [ ]:
best_params_per_model['lr']

In [ ]:
print(best_params_per_model['lr'].to_latex())